In [1]:
import os

In [2]:
%pwd

'/Users/rajusubba/Documents/End-to-End MLOps/customer-churn-project/research'

In [4]:
os.chdir("/Users/rajusubba/Documents/End-to-End MLOps/customer-churn-project")
%pwd

'/Users/rajusubba/Documents/End-to-End MLOps/customer-churn-project'

In [5]:
import pandas as pd
df = pd.read_csv("artifacts/data_ingestion/raw_snapshot.csv")
df.head()

,customerid,surname,creditscore,geography,gender,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,exited
0,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customerid       10000 non-null  int64  
 1   surname          10000 non-null  object 
 2   creditscore      10000 non-null  int64  
 3   geography        10000 non-null  object 
 4   gender           10000 non-null  object 
 5   age              10000 non-null  int64  
 6   tenure           10000 non-null  int64  
 7   balance          10000 non-null  float64
 8   numofproducts    10000 non-null  int64  
 9   hascrcard        10000 non-null  int64  
 10  isactivemember   10000 non-null  int64  
 11  estimatedsalary  10000 non-null  float64
 12  exited           10000 non-null  int64  
dtypes: float64(2), int64(8), object(3)
memory usage: 1015.8+ KB


In [7]:
df.isnull().sum()

customerid         0
surname            0
creditscore        0
geography          0
gender             0
age                0
tenure             0
balance            0
numofproducts      0
hascrcard          0
isactivemember     0
estimatedsalary    0
exited             0
dtype: int64

In [8]:
df.shape

(10000, 13)

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataValidationConfig:
    root_dir: Path
    data_dir: Path
    STATUS_FILE: str
    all_schema: dict

In [10]:
from src.customerchurn.constants import *
from src.customerchurn.utils.common import read_yaml, create_directories

In [13]:
class ConfigurationManager:
    def __init__(self, 
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH,
                 schema_file_path = SCHEMA_FILE_PATH,
                 ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_validation_config(self) -> DataValidationConfig:
        validation_config = self.config.data_validation
        schema = self.schema.COLUMNS
        
        create_directories([validation_config.root_dir])
        
        data_validation_config = DataValidationConfig(
            root_dir = validation_config.root_dir,
            STATUS_FILE = validation_config.STATUS_FILE,
            data_dir = validation_config.data_dir,
            all_schema = schema
        )
        return data_validation_config
        

In [14]:
import os
from src.customerchurn.logging.logger import logging



In [17]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config
        
    def validate_all_columns(self) -> bool:
        try:
            validation_status = None
            df = pd.read_csv(self.config.data_dir)
            all_cols = list(df.columns)
            all_schema_cols = list(self.config.all_schema.keys())
            
            for col in all_schema_cols:
                if col not in all_schema_cols:
                    validation_status = False
                    with open(self.config.STATUS_FILE, "w") as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, "w") as f:
                        f.write(f"Validation status: {validation_status}")
            return validation_status
        except Exception as e:
            logging.error(f"Error occured in validate_all_columns method of DataValidation class. Error: {e}")
            raise e

In [18]:
try:
    config_manager = ConfigurationManager()
    data_validation_config = config_manager.get_data_validation_config()
    
    data_validation = DataValidation(config = data_validation_config)
    status = data_validation.validate_all_columns()
    print(f"Data validation status: {status}")
except Exception as e:
    logging.error(f"Error occured in Data Validation. Error: {e}")
    raise e

[2026-02-04 10:22:24,978: INFO: common: YAML file config/config.yaml loaded successfully.]
[2026-02-04 10:22:24,980: INFO: common: YAML file params.yaml loaded successfully.]
[2026-02-04 10:22:24,982: INFO: common: YAML file schema.yaml loaded successfully.]
[2026-02-04 10:22:24,983: INFO: common: Directory created at: artifacts]
[2026-02-04 10:22:24,983: INFO: common: Directory created at: artifacts/data_validation]
Data validation status: True
